# Test Best Models

Test notebook to generate the statistics of the different models found with AMS. First load the required libraries

In [9]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K
from keras.models import model_from_json

from sklearn.model_selection import KFold
from scipy import stats

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler(data_scaler=None):

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features,
                                       max_rul, window_size, window_stride, data_scaler=data_scaler)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps

### Load models and evaluate model

In [4]:
def load_model(model_file, weights_file="", problem_type=1):
    
    p_type = ""
    
    # load json and create model
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("Loaded model from disk") 
        
    #Load weights if specified
    if weights_file != "":
        # load weights into new model
        loaded_model.load_weights(weights_file)
        print("Loaded weights from disk") 
    else:
        print("Model needs training")
        
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        p_type = "regression"
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        p_type = "classification"
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    print("Created model for " + p_type + " with loss function " + lossFunction)

    return loaded_model
        
        

### Load each of the models and test them

In [ ]:
best_model_folder = 'best_models'
test_sets = {'cifar10':(CIFAR10DataHandler, None, 2), 
             'cmapss':(cmaps_dhandler, MinMaxScaler(feature_range=(-1, 1)), 1), 
             'mnist':((MNISTDataHandler), None, 2)}
alpha_folders = ['alpha0.6', 'alpha0.8', 'alpha1']

In [14]:
results = {}

k = 2  #For 10-fold cross validation

model_file_name = 'bestModel_global.json'
weights_file_name = 'bestModel_global.h5'

for dataset in test_sets.keys():
    for alphaValue in alpha_folders:
        
        results_key = dataset + "_" +  alphaValue
        
        evaluations_cv = np.zeros(k)
        evaluations_test = np.zeros(k)
        
        model_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + model_file_name
        
        if weights_file_name != "":
            weights_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + weights_file_name
        
        dhandler, data_scaler, problem_type = test_sets[dataset]

        #model = load_model(model_location, weights_location, problem_type)   
        
        data_handler = dhandler(data_scaler=data_scaler)
        data_handler.load_data(verbose = 1, )
        data_handler.print_data()
        
        folds = list(KFold(n_splits=k, shuffle=True).split(data_handler.X_train))
        
        print('Validation on model:' + model_location)
        
        for j, (train_idx, val_idx) in enumerate(folds):

            print('\nExperiment on Fold ', j)
            
            K.clear_session()  #Clear the previous tensorflow graph 

            X_train_cv = data_handler.X_train[train_idx]
            y_train_cv = data_handler.y_train[train_idx]
            X_valid_cv = data_handler.X_train[val_idx]
            y_valid_cv = data_handler.y_train[val_idx]

            model = load_model(model_location, "", problem_type)
            model.summary()

            model.fit(X_train_cv, y_train_cv, batch_size=512, epochs=100, verbose=1)

            evaluation_cv = model.evaluate(X_valid_cv, y_valid_cv)
            evaluation_test = model.evaluate(data_handler.X_test, data_handler.y_test)

            evaluations_cv[j] = evaluation_cv[1]
            evaluations_test[j] = evaluation_test[1]
            
        results[results_key] = (evaluations_cv, evaluations_test)
        

Loading data. Cros-Validation ratio 0
Printing shapes

Training data (X, y)
(50000, 3072)
(50000, 10)
Testing data (X, y)
(10000, 3072)
(10000, 10)
Printing first 5 elements

Training data (X, y)
[[0.23137255 0.24313726 0.24705882 ... 0.48235294 0.36078432 0.28235295]
 [0.6039216  0.69411767 0.73333335 ... 0.56078434 0.52156866 0.5647059 ]
 [1.         1.         1.         ... 0.3137255  0.3372549  0.32941177]
 [0.10980392 0.09803922 0.03921569 ... 0.28235295 0.25490198 0.18039216]
 [0.6666667  0.7058824  0.7764706  ... 0.28627452 0.3019608  0.3137255 ]]
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Testing data (X, y)
[[0.61960787 0.4392157  0.19215687 ... 0.08235294 0.2627451  0.43137255]
 [0.92156863 0.92156863 0.92156863 ... 0.7294118  0.78431374 0.78039217]
 [0.61960787 0.74509805 0.87058824 ... 0.02745098 0.03137255 0.02745098]
 [0.60784316 0.6117647  0.5843137

25000/25000 [==============================] - 0s 8us/step - loss: 0.9642 - acc: 0.6608
Epoch 60/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.9579 - acc: 0.6623
Epoch 61/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.9468 - acc: 0.6660
Epoch 62/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.9387 - acc: 0.6674
Epoch 63/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.9154 - acc: 0.6784
Epoch 64/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.9408 - acc: 0.6675
Epoch 65/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.9167 - acc: 0.6769
Epoch 66/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.9000 - acc: 0.6833
Epoch 67/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.8862 - acc: 0.6904
Epoch 68/100
25000/25000 [==============================] - 0s 8us/step - loss: 0.8750 - acc: 0.6902
Epo

Epoch 30/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.2577 - acc: 0.5548
Epoch 31/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.2497 - acc: 0.5573
Epoch 32/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.2289 - acc: 0.5658
Epoch 33/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.2245 - acc: 0.5681
Epoch 34/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.2114 - acc: 0.5720
Epoch 35/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.1880 - acc: 0.5800
Epoch 36/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.1751 - acc: 0.5824
Epoch 37/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.1696 - acc: 0.5882
Epoch 38/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.1535 - acc: 0.5915
Epoch 39/100
25000/25000 [==============================] - 0s 8us/step - loss: 1.1379 - ac

Created model for classification with loss function categorical_crossentropy
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 240)               737520    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2410      
Total params: 739,930
Trainable params: 739,930
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25000/25000 [==============================] - 0s 12us/step - loss: 2.1024 - acc: 0.2377
Epoch 2/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.9084 - acc: 0.3235
Epoch 3/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.8401 - acc: 0.3500
Epoch 4/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.7992 - acc: 0.3690
Epoch 5/100
25000/25000 [=============================

25000/25000 [==============================] - 0s 7us/step - loss: 1.0066 - acc: 0.6582
Epoch 76/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9979 - acc: 0.6589
Epoch 77/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9935 - acc: 0.6640
Epoch 78/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.0005 - acc: 0.6590
Epoch 79/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9767 - acc: 0.6678
Epoch 80/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9831 - acc: 0.6653
Epoch 81/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9692 - acc: 0.6722
Epoch 82/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9621 - acc: 0.6744
Epoch 83/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9599 - acc: 0.6766
Epoch 84/100
25000/25000 [==============================] - 0s 7us/step - loss: 0.9641 - acc: 0.6738
Epo

...

30000/30000 [==============================] - 0s 4us/step - loss: 0.0941 - acc: 0.9748
Epoch 55/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0930 - acc: 0.9744
Epoch 56/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0910 - acc: 0.9752
Epoch 57/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0896 - acc: 0.9766
Epoch 58/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0881 - acc: 0.9759
Epoch 59/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0867 - acc: 0.9766
Epoch 60/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0848 - acc: 0.9769
Epoch 61/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0842 - acc: 0.9774
Epoch 62/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0825 - acc: 0.9778
Epoch 63/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0812 - acc: 0.9781
Epo

30000/30000 [==============================] - 0s 4us/step - loss: 0.1461 - acc: 0.9582
Epoch 27/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1429 - acc: 0.9594
Epoch 28/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1395 - acc: 0.9598
Epoch 29/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1367 - acc: 0.9607
Epoch 30/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1348 - acc: 0.9610
Epoch 31/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1323 - acc: 0.9610
Epoch 32/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1286 - acc: 0.9632
Epoch 33/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1274 - acc: 0.9632
Epoch 34/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1247 - acc: 0.9643
Epoch 35/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1224 - acc: 0.9648
Epo

Epoch 1/100
30000/30000 [==============================] - 0s 8us/step - loss: 1.3167 - acc: 0.6391
Epoch 2/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.4374 - acc: 0.8814
Epoch 3/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.3295 - acc: 0.9054
Epoch 4/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.2890 - acc: 0.9162
Epoch 5/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.2605 - acc: 0.9246
Epoch 6/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.2418 - acc: 0.9294
Epoch 7/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.2246 - acc: 0.9346
Epoch 8/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.2111 - acc: 0.9387
Epoch 9/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1980 - acc: 0.9415
Epoch 10/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.1886 - acc: 0.9444

30000/30000 [==============================] - 0s 4us/step - loss: 0.0350 - acc: 0.9908
Epoch 83/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0335 - acc: 0.9925
Epoch 84/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0342 - acc: 0.9911
Epoch 85/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0334 - acc: 0.9917
Epoch 86/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0328 - acc: 0.9914
Epoch 87/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0321 - acc: 0.9919
Epoch 88/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0314 - acc: 0.9923
Epoch 89/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0305 - acc: 0.9930
Epoch 90/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0298 - acc: 0.9935
Epoch 91/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0294 - acc: 0.9930
Epo

30000/30000 [==============================] - 0s 4us/step - loss: 0.0735 - acc: 0.9786
Epoch 54/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0723 - acc: 0.9783
Epoch 55/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0716 - acc: 0.9795
Epoch 56/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0701 - acc: 0.9788
Epoch 57/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0684 - acc: 0.9798
Epoch 58/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0675 - acc: 0.9801
Epoch 59/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0673 - acc: 0.9806
Epoch 60/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0639 - acc: 0.9810
Epoch 61/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0628 - acc: 0.9817
Epoch 62/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0628 - acc: 0.9821
Epo

### Print the stats

In [16]:
for key in results.keys():
    
    print("\nStats for: ")
    print(key)
    evaluations_cv, evaluations_test = results[key]
    
    print("\nCrossVal stats: ")
    print(evaluations_cv)
    print(stats.describe(evaluations_cv))
    
    print("\nTest stats: ")
    print(stats.describe(evaluations_test))


Stats for: 
cifar10_alpha0.6

CrossVal stats: 
[0.47508 0.4686 ]
DescribeResult(nobs=2, minmax=(0.4686, 0.47508), mean=0.47184000000000004, variance=2.0995199999999907e-05, skewness=-2.5682023229530742e-14, kurtosis=-2.0)

Test stats: 
DescribeResult(nobs=2, minmax=(0.473, 0.4747), mean=0.47385, variance=1.4450000000000592e-06, skewness=0.0, kurtosis=-2.0)

Stats for: 
cifar10_alpha0.8

CrossVal stats: 
[0.4906  0.48684]
DescribeResult(nobs=2, minmax=(0.48684, 0.4906), mean=0.48872, variance=7.068799999999946e-06, skewness=0.0, kurtosis=-2.0)

Test stats: 
DescribeResult(nobs=2, minmax=(0.493, 0.4948), mean=0.4939, variance=1.620000000000043e-06, skewness=0.0, kurtosis=-2.0)

Stats for: 
cifar10_alpha1

CrossVal stats: 
[0.48828 0.49932]
DescribeResult(nobs=2, minmax=(0.48828, 0.49932), mean=0.4938, variance=6.094079999999994e-05, skewness=-1.5029220755522348e-14, kurtosis=-2.0)

Test stats: 
DescribeResult(nobs=2, minmax=(0.49, 0.4945), mean=0.49224999999999997, variance=1.0125000000